In [1]:
from utils import *
data_path = 'data/'
oof_path =  data_path+'oof/'
submission_path = data_path+ 'submission/'

In [2]:
def get_X_y_ensembling(df,clip=20):
    y = np.clip(df.target.values,0,clip)
    X = df.drop(['target','date_block_num'],axis=1)
    return X,y
def get_oof_pickle(filename,clip=20):
    return np.clip(pd.read_pickle(oof_path+filename),0,clip)

In [32]:
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import ElasticNet,Ridge,LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor


# Generate ensembling training set

In [3]:
# lgb_oof = get_oof_pickle('lgb_best.pickle')

# xgb2_oof = get_oof_pickle('xgb2.pickle')
# xgb3_oof = get_oof_pickle('xgb3.pickle')

# # fast_oof = get_oof_pickle('fastai1000500.pickle')

# all_data = get_all_data(data_path,'new_sales_lag_after12.pickle')

# all_data.target = np.clip(all_data.target.values,0,20)

# temp = [lgb_oof,xgb2_oof,xgb3_oof,
# #         fast_oof,
#         lgb_oof-xgb2_oof,lgb_oof-xgb3_oof,
# #         lgb_oof-fast_oof,
#         xgb2_oof-xgb3_oof,
# #         xgb2_oof-fast_oof,xgb3_oof-fast_oof,
# #         np.mean(np.stack([lgb_oof,xgb2_oof,xgb3_oof,fast_oof],axis=1),axis=1)
#         np.mean(np.stack([lgb_oof,xgb2_oof,xgb3_oof],axis=1),axis=1)]

# all28 = all_data[all_data.date_block_num>=28].copy()

# all_oof = np.stack(temp+[all28.target_lag_1,all28.date_block_num,all28.target],axis=1)

# all_oof_df = pd.DataFrame(all_oof,columns=['lgb','xgb2','xgb3',
# #                                            'fast',
#                                            'lgb_xgb2','lgb_xgb3',
# #                                            'lgb_fast',
#                                            'xgb2_xgb3',
# #                                            'xgb2_fast','xgb3_fast',
#                                            'oof_mean','target_lag_1','date_block_num','target'])

# all_oof_df.to_pickle(oof_path+'ensembling_nofast.pickle')

In [3]:
all_oof_df= pd.read_pickle(oof_path+'ensembling_nofast.pickle')
all_oof_df.head().T

,0,1,2,3,4
lgb,0.378975,0.077147,0.107809,0.580430,1.194422
xgb2,0.451092,0.100030,0.142697,0.662918,1.272898
xgb3,0.477408,0.096444,0.137257,0.686700,1.174996
lgb_xgb2,-0.072118,-0.022882,-0.034888,-0.082488,-0.078476
lgb_xgb3,-0.098434,-0.019297,-0.029448,-0.106270,0.019426
xgb2_xgb3,-0.026316,0.003586,0.005440,-0.023782,0.097902
oof_mean,0.435825,0.091207,0.129255,0.643350,1.214105
target_lag_1,0.000000,0.000000,0.000000,1.000000,3.000000
date_block_num,28.000000,28.000000,28.000000,28.000000,28.000000
target,2.000000,1.000000,1.000000,2.000000,2.000000


In [5]:
cv = get_cv_idxs(all_oof_df,29,33)

# Generate test set

In [18]:
# This all should be 20-clipped already
lgb_pred = pd.read_csv(submission_path+'coursera_tuned_lightgbm_basic_6folds.csv').item_cnt_month
#xgb grow with max leaf node
xgb2_pred = pd.read_csv(submission_path+'tuned_xgb_basicfeatures_6folds_8136.csv').item_cnt_month
#xgb grow with max depth
xgb3_pred = pd.read_csv(submission_path+'tuned_xgb_basicfeatures_6folds_8126.csv').item_cnt_month

target_lag_1 = pd.read_csv(data_path+'test_lag.csv').target_lag_1

#fastai
#???


temp = [lgb_pred,xgb2_pred,xgb3_pred,
#         fast_oof,
        lgb_pred-xgb2_pred,lgb_pred-xgb3_pred,
#         lgb_oof-fast_oof,
        xgb2_pred-xgb3_pred,
#         xgb2_oof-fast_oof,xgb3_oof-fast_oof,
        np.mean(np.stack([lgb_pred,xgb2_pred,xgb3_pred],axis=1),axis=1),target_lag_1]

temp = np.stack(temp,axis=1)
temp.shape

test_df = pd.DataFrame(temp,columns=['lgb','xgb2','xgb3',
#                                            'fast',
                                           'lgb_xgb2','lgb_xgb3',
#                                            'lgb_fast',
                                           'xgb2_xgb3',
#                                            'xgb2_fast','xgb3_fast',
                                           'oof_mean','target_lag_1'])

test_df.to_csv(data_path+'ensembling_test_nofast.csv',index=False)

In [19]:
test_df= pd.read_csv(data_path+'ensembling_test_nofast.csv')
test_df.head()

,lgb,xgb2,xgb3,lgb_xgb2,lgb_xgb3,xgb2_xgb3,oof_mean,target_lag_1
0,0.596835,0.493997,0.635656,0.102838,-0.038821,-0.141659,0.575496,0.0
1,0.252161,0.248587,0.277100,0.003574,-0.024939,-0.028513,0.259283,0.0
2,1.002808,1.003845,1.193923,-0.001037,-0.191115,-0.190079,1.066859,1.0
3,0.361629,0.311269,0.362728,0.050360,-0.001099,-0.051459,0.345209,0.0
4,1.986176,3.029626,2.800084,-1.043451,-0.813908,0.229542,2.605295,0.0


# Simple average

In [26]:
# final_pred = lgb_pred*0.8 + xgb2_pred*0.1+xgb3_pred*0.1

# get_submission(final_pred,'ensembling_average_8.1.1'); # 0.90876 leaderboard

# Linear regression

In [60]:
X,y = get_X_y_ensembling(all_oof_df)

# elastic = ElasticNetCV(random_state=1402,l1_ratio=[.01,0.05,.1, .5, .7, .9, .95, .99, 1],cv =cv,verbose=10,n_jobs=2)
# elastic = ElasticNet(random_state=1402)
lr = LinearRegression()
params={
#     'l1_ratio': [0.0,.001,.005,.01,0.05,.1, .5,.6, .7, .8,.9, 1]
    'fit_intercept':[True,False],
    'normalize':[True,False]
}
clf=  GridSearchCV(lr,params,cv=cv,scoring = make_scorer(root_mean_squared_error,greater_is_better=False),verbose=True,n_jobs=-1)

In [63]:
clf.fit(X,y)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   17.5s finished


GridSearchCV(cv=array([[array([     0,      1, ..., 232450, 232451], dtype=int64),
        array([232452, 232453, ..., 456738, 456739], dtype=int64)],
       [array([     0,      1, ..., 456738, 456739], dtype=int64),
        array([456740, 456741, ..., 685627, 685628], dtype=int64)],
       [array([     0,     ...dtype=int64),
        array([1118820, 1118821, ..., 1356990, 1356991], dtype=int64)]], dtype=object),
       error_score='raise',
       estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'fit_intercept': [True, False], 'normalize': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(root_mean_squared_error, greater_is_better=False),
       verbose=True)

In [65]:
print(clf.best_params_)
print(clf.best_score_)

{'fit_intercept': False, 'normalize': True}
-0.806418475542


In [67]:
cv_result=pd.DataFrame(clf.cv_results_)
cv_result.T

C:\Users\qtran\AppData\Local\Continuum\Miniconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\qtran\AppData\Local\Continuum\Miniconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\qtran\AppData\Local\Continuum\Miniconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_arg

,0,1,2,3
mean_fit_time,0.291597,0.763952,0.85235,0.85195
mean_score_time,0.00471992,0.00955987,0.0312397,0.0197998
mean_test_score,-0.806568,-0.806467,-0.806418,-0.806418
mean_train_score,-0.789094,-0.789089,-0.789239,-0.789239
param_fit_intercept,True,True,False,False
param_normalize,True,False,True,False
params,"{'fit_intercept': True, 'normalize': True}","{'fit_intercept': True, 'normalize': False}","{'fit_intercept': False, 'normalize': True}","{'fit_intercept': False, 'normalize': False}"
rank_test_score,4,3,1,1
split0_test_score,-0.777867,-0.777262,-0.77736,-0.77736
split0_train_score,-0.828671,-0.828645,-0.828686,-0.828686


In [57]:
# cv_result=pd.DataFrame(clf.cv_results_)
# cv_result

C:\Users\qtran\AppData\Local\Continuum\Miniconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\qtran\AppData\Local\Continuum\Miniconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\qtran\AppData\Local\Continuum\Miniconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_arg

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_l1_ratio,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,37.172250,0.01680,-0.844334,-0.826653,0,{'l1_ratio': 0.0},1,-0.807423,-0.861509,-0.742919,...,-0.818949,-0.804664,-0.913504,-0.805938,-0.935918,-0.827148,27.141542,0.008103,0.072075,0.020890
1,1.992440,0.01260,-0.844561,-0.826876,0.001,{'l1_ratio': 0.001},2,-0.807604,-0.861729,-0.743154,...,-0.819214,-0.804886,-0.913784,-0.806171,-0.936101,-0.827380,0.861523,0.006406,0.072073,0.020886
2,2.103959,0.01580,-0.845459,-0.827777,0.005,{'l1_ratio': 0.005},3,-0.808335,-0.862619,-0.744112,...,-0.820314,-0.805788,-0.914761,-0.807103,-0.936841,-0.828318,0.603955,0.013197,0.072032,0.020872
3,2.109673,0.00972,-0.846567,-0.828914,0.01,{'l1_ratio': 0.01},4,-0.809259,-0.863749,-0.745388,...,-0.821718,-0.806937,-0.915982,-0.808287,-0.937593,-0.829465,0.583238,0.005761,0.071909,0.020854
4,1.675500,0.00312,-0.856498,-0.838663,0.05,{'l1_ratio': 0.05},5,-0.819146,-0.872932,-0.756281,...,-0.833623,-0.816919,-0.926504,-0.818564,-0.944320,-0.839425,0.415466,0.006240,0.070773,0.020485
5,2.265339,0.02636,-0.870917,-0.852805,0.1,{'l1_ratio': 0.1},6,-0.834158,-0.886029,-0.771680,...,-0.850259,-0.831466,-0.941630,-0.833504,-0.954595,-0.853917,0.978752,0.029936,0.069359,0.019863
6,0.905838,0.03540,-1.038082,-1.022502,0.5,{'l1_ratio': 0.5},7,-1.017234,-1.057547,-0.946900,...,-1.022024,-0.999357,-1.110829,-1.003214,-1.093021,-1.023134,0.390283,0.037734,0.059130,0.020890
7,0.990000,0.01460,-1.042693,-1.026014,0.6,{'l1_ratio': 0.6},8,-1.021657,-1.059743,-0.950955,...,-1.026340,-1.002996,-1.115963,-1.007502,-1.098128,-1.027867,0.101398,0.006946,0.059556,0.020244
8,1.056400,0.05648,-1.047587,-1.030137,0.7,{'l1_ratio': 0.7},9,-1.026350,-1.062327,-0.955168,...,-1.030886,-1.007264,-1.121407,-1.012535,-1.103676,-1.033424,0.228955,0.059497,0.060063,0.019519
9,1.189595,0.02332,-1.052774,-1.034889,0.8,{'l1_ratio': 0.8},10,-1.031316,-1.065305,-0.959543,...,-1.035669,-1.012180,-1.127173,-1.018338,-1.109682,-1.039836,0.378167,0.035289,0.060655,0.018732


## Prediction

In [ ]:
X,y = get_X_y_ensembling(all_oof_df)

lr = LinearRegression(fit_intercept=False,n_jobs=-1)

lr.fit(X,y)

X.head()

test_pred =  lr.predict(test_df)

pd.Series(test_pred).describe()

get_submission(test_pred,'ensembling_lr'); # 0.90982 LB

# Ridge

In [26]:
X,y = get_X_y_ensembling(all_oof_df)

# elastic = ElasticNetCV(random_state=1402,l1_ratio=[.01,0.05,.1, .5, .7, .9, .95, .99, 1],cv =cv,verbose=10,n_jobs=2)
# elastic = ElasticNet(random_state=1402)
lr = Ridge(random_state=1402)
params={
#     'l1_ratio': [0.0,.001,.005,.01,0.05,.1, .5,.6, .7, .8,.9, 1]
    
    'alpha':[0.0,.001,.005,.01,0.05,.1, .5,.6, .7, .8,.9, 1],
    'fit_intercept': [False],
    'solver':['svd','cholesky','sparse_cg','lsqr','sag']
    
}
clf=  GridSearchCV(lr,params,cv=cv,scoring = make_scorer(root_mean_squared_error,greater_is_better=False),verbose=True,n_jobs=-1)

In [27]:
clf.fit(X,y)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   23.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  6.9min finished


GridSearchCV(cv=array([[array([     0,      1, ..., 232450, 232451], dtype=int64),
        array([232452, 232453, ..., 456738, 456739], dtype=int64)],
       [array([     0,      1, ..., 456738, 456739], dtype=int64),
        array([456740, 456741, ..., 685627, 685628], dtype=int64)],
       [array([     0,     ...dtype=int64),
        array([1118820, 1118821, ..., 1356990, 1356991], dtype=int64)]], dtype=object),
       error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=1402, solver='auto', tol=0.001),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'alpha': [0.0, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 0.6, 0.7, 0.8, 0.9, 1], 'fit_intercept': [False], 'solver': ['svd', 'cholesky', 'sparse_cg', 'lsqr', 'sag']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(root_mean_squared_error, greater_is_better=False),
       verbose=True)

In [28]:
print(clf.best_params_)
print(clf.best_score_)

{'alpha': 0.0, 'fit_intercept': False, 'solver': 'sag'}
-0.806023806956


In [30]:
cv_result=pd.DataFrame(clf.cv_results_).sort_values('rank_test_score')
cv_result.T

C:\Users\qtran\AppData\Local\Continuum\Miniconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\qtran\AppData\Local\Continuum\Miniconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\qtran\AppData\Local\Continuum\Miniconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_arg

,4,9,14,19,24,29,34,39,44,49,...,42,37,32,27,22,17,12,7,2,0
mean_fit_time,45.8915,44.747,44.9763,45.6628,46.0089,44.8395,44.3174,44.5545,44.9972,45.1884,...,1.07843,1.02219,1.03907,0.885629,1.09659,1.04619,0.922268,1.1824,0.579433,0.243958
mean_score_time,0.00963993,0.00471992,0.0124798,0.0173999,0.0218397,0.00451994,0.0124799,0.00935984,0.00472002,0.0117599,...,0.0532392,0.0719589,0.0587995,0.0473994,0.0530392,0.0467995,0.0499193,0.0374,0.0343195,0.00803981
mean_test_score,-0.806024,-0.806024,-0.806024,-0.806024,-0.806024,-0.806024,-0.806024,-0.806024,-0.806024,-0.806024,...,-0.80647,-0.80647,-0.80647,-0.80647,-0.80647,-0.80647,-0.80647,-0.80647,-0.80647,-0.810413
mean_train_score,-0.789276,-0.789276,-0.789276,-0.789276,-0.789276,-0.789276,-0.789276,-0.789276,-0.789276,-0.789276,...,-0.789243,-0.789243,-0.789243,-0.789243,-0.789243,-0.789243,-0.789243,-0.789243,-0.789243,-0.796948
param_alpha,0,0.001,0.005,0.01,0.05,0.1,0.5,0.6,0.7,0.8,...,0.7,0.6,0.5,0.1,0.05,0.01,0.005,0.001,0,0
param_fit_intercept,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
param_solver,sag,sag,sag,sag,sag,sag,sag,sag,sag,sag,...,sparse_cg,sparse_cg,sparse_cg,sparse_cg,sparse_cg,sparse_cg,sparse_cg,sparse_cg,sparse_cg,svd
params,"{'alpha': 0.0, 'fit_intercept': False, 'solver...","{'alpha': 0.001, 'fit_intercept': False, 'solv...","{'alpha': 0.005, 'fit_intercept': False, 'solv...","{'alpha': 0.01, 'fit_intercept': False, 'solve...","{'alpha': 0.05, 'fit_intercept': False, 'solve...","{'alpha': 0.1, 'fit_intercept': False, 'solver...","{'alpha': 0.5, 'fit_intercept': False, 'solver...","{'alpha': 0.6, 'fit_intercept': False, 'solver...","{'alpha': 0.7, 'fit_intercept': False, 'solver...","{'alpha': 0.8, 'fit_intercept': False, 'solver...",...,"{'alpha': 0.7, 'fit_intercept': False, 'solver...","{'alpha': 0.6, 'fit_intercept': False, 'solver...","{'alpha': 0.5, 'fit_intercept': False, 'solver...","{'alpha': 0.1, 'fit_intercept': False, 'solver...","{'alpha': 0.05, 'fit_intercept': False, 'solve...","{'alpha': 0.01, 'fit_intercept': False, 'solve...","{'alpha': 0.005, 'fit_intercept': False, 'solv...","{'alpha': 0.001, 'fit_intercept': False, 'solv...","{'alpha': 0.0, 'fit_intercept': False, 'solver...","{'alpha': 0.0, 'fit_intercept': False, 'solver..."
rank_test_score,1,2,3,4,5,6,7,8,9,10,...,51,52,53,54,55,56,57,58,59,60
split0_test_score,-0.774951,-0.774951,-0.774951,-0.774951,-0.774951,-0.774951,-0.774951,-0.774951,-0.774951,-0.774951,...,-0.77736,-0.77736,-0.77736,-0.77736,-0.77736,-0.77736,-0.77736,-0.77736,-0.77736,-0.775314


In [31]:
X,y = get_X_y_ensembling(all_oof_df)

params={'alpha': 0.0, 'fit_intercept': False, 'solver': 'sag','random_state':1402}
lr = Ridge(**params)

lr.fit(X,y)

test_pred =  lr.predict(test_df)

pd.Series(test_pred).describe()

get_submission(test_pred,'ensembling_ridge');

# Last attempt: Shallow random forest

In [38]:
X,y = get_X_y_ensembling(all_oof_df)

# elastic = ElasticNetCV(random_state=1402,l1_ratio=[.01,0.05,.1, .5, .7, .9, .95, .99, 1],cv =cv,verbose=10,n_jobs=2)
# elastic = ElasticNet(random_state=1402)
rf = RandomForestRegressor(random_state=1402)
params = {'max_depth': [3,4,5,6,7], 'n_estimators': [7,10,15,20,25], 'max_features': [.4,.5,0.7,.9], 'oob_score': [True]}
clf=  GridSearchCV(rf,params,cv=cv,scoring = make_scorer(root_mean_squared_error,greater_is_better=False),verbose=True,n_jobs=-1)

In [39]:
clf.fit(X,y)
print(clf.best_params_)
print(clf.best_score_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed: 30.8min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 39.5min finished


{'max_depth': 5, 'max_features': 0.4, 'n_estimators': 25, 'oob_score': True}
-0.807331441099


C:\Users\qtran\AppData\Local\Continuum\Miniconda3\lib\site-packages\sklearn\ensemble\forest.py:724: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "


In [43]:
for i in pd.DataFrame(clf.cv_results_).sort_values('rank_test_score').params[:10]:
    print(i)

{'max_depth': 5, 'max_features': 0.4, 'n_estimators': 25, 'oob_score': True}
{'max_depth': 5, 'max_features': 0.4, 'n_estimators': 20, 'oob_score': True}
{'max_depth': 6, 'max_features': 0.4, 'n_estimators': 10, 'oob_score': True}
{'max_depth': 5, 'max_features': 0.7, 'n_estimators': 15, 'oob_score': True}
{'max_depth': 6, 'max_features': 0.4, 'n_estimators': 15, 'oob_score': True}
{'max_depth': 6, 'max_features': 0.4, 'n_estimators': 7, 'oob_score': True}
{'max_depth': 5, 'max_features': 0.5, 'n_estimators': 25, 'oob_score': True}
{'max_depth': 6, 'max_features': 0.4, 'n_estimators': 25, 'oob_score': True}
{'max_depth': 5, 'max_features': 0.5, 'n_estimators': 20, 'oob_score': True}
{'max_depth': 5, 'max_features': 0.4, 'n_estimators': 15, 'oob_score': True}


C:\Users\qtran\AppData\Local\Continuum\Miniconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\qtran\AppData\Local\Continuum\Miniconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\qtran\AppData\Local\Continuum\Miniconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_arg

In [44]:
cv_result=pd.DataFrame(clf.cv_results_).sort_values('rank_test_score')
cv_result.T

C:\Users\qtran\AppData\Local\Continuum\Miniconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\qtran\AppData\Local\Continuum\Miniconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\qtran\AppData\Local\Continuum\Miniconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_arg

,44,43,61,52,62,60,49,64,48,42,...,75,96,2,91,1,16,90,0,95,15
mean_fit_time,41.1944,33.2182,18.7152,37.2436,28.5904,13.5068,51.6172,47.19,42.0934,24.5236,...,27.2056,45.3174,15.5134,32.064,9.9432,20.8878,22.7178,4.9206,31.4252,15.401
mean_score_time,0.3176,0.247119,0.1126,0.153,0.1722,0.0854001,0.2546,0.301,0.2034,0.1594,...,0.16892,0.1358,0.093,0.1338,0.073,0.0678001,0.099,0.041,0.1152,0.0614
mean_test_score,-0.807331,-0.807551,-0.807643,-0.80776,-0.807813,-0.807822,-0.807828,-0.807867,-0.80789,-0.807921,...,-0.81293,-0.813089,-0.813153,-0.813189,-0.81331,-0.813587,-0.813594,-0.813903,-0.81417,-0.814235
mean_train_score,-0.770775,-0.770977,-0.763573,-0.769853,-0.76302,-0.764323,-0.77021,-0.762324,-0.770248,-0.77114,...,-0.760494,-0.748664,-0.787206,-0.749744,-0.787189,-0.786328,-0.750645,-0.787873,-0.749389,-0.787111
param_max_depth,5,5,6,5,6,6,5,6,5,5,...,6,7,3,7,3,3,7,3,7,3
param_max_features,0.4,0.4,0.4,0.7,0.4,0.4,0.5,0.4,0.5,0.4,...,0.9,0.9,0.4,0.7,0.4,0.9,0.7,0.4,0.9,0.9
param_n_estimators,25,20,10,15,15,7,25,25,20,15,...,7,10,15,10,10,10,7,7,7,7
param_oob_score,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
params,"{'max_depth': 5, 'max_features': 0.4, 'n_estim...","{'max_depth': 5, 'max_features': 0.4, 'n_estim...","{'max_depth': 6, 'max_features': 0.4, 'n_estim...","{'max_depth': 5, 'max_features': 0.7, 'n_estim...","{'max_depth': 6, 'max_features': 0.4, 'n_estim...","{'max_depth': 6, 'max_features': 0.4, 'n_estim...","{'max_depth': 5, 'max_features': 0.5, 'n_estim...","{'max_depth': 6, 'max_features': 0.4, 'n_estim...","{'max_depth': 5, 'max_features': 0.5, 'n_estim...","{'max_depth': 5, 'max_features': 0.4, 'n_estim...",...,"{'max_depth': 6, 'max_features': 0.9, 'n_estim...","{'max_depth': 7, 'max_features': 0.9, 'n_estim...","{'max_depth': 3, 'max_features': 0.4, 'n_estim...","{'max_depth': 7, 'max_features': 0.7, 'n_estim...","{'max_depth': 3, 'max_features': 0.4, 'n_estim...","{'max_depth': 3, 'max_features': 0.9, 'n_estim...","{'max_depth': 7, 'max_features': 0.7, 'n_estim...","{'max_depth': 3, 'max_features': 0.4, 'n_estim...","{'max_depth': 7, 'max_features': 0.9, 'n_estim...","{'max_depth': 3, 'max_features': 0.9, 'n_estim..."
rank_test_score,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100


In [37]:
# for i in pd.DataFrame(clf.cv_results_).sort_values('rank_test_score').params:
#     print(i)

{'max_depth': 4, 'max_features': 0.5, 'n_estimators': 20, 'oob_score': True}
{'max_depth': 4, 'max_features': 0.7, 'n_estimators': 15, 'oob_score': True}
{'max_depth': 4, 'max_features': 0.7, 'n_estimators': 20, 'oob_score': True}
{'max_depth': 4, 'max_features': 0.5, 'n_estimators': 15, 'oob_score': True}
{'max_depth': 4, 'max_features': 0.7, 'n_estimators': 13, 'oob_score': True}
{'max_depth': 4, 'max_features': 0.7, 'n_estimators': 7, 'oob_score': True}
{'max_depth': 4, 'max_features': 0.5, 'n_estimators': 13, 'oob_score': True}
{'max_depth': 4, 'max_features': 0.7, 'n_estimators': 10, 'oob_score': True}
{'max_depth': 4, 'max_features': 0.7, 'n_estimators': 5, 'oob_score': True}
{'max_depth': 4, 'max_features': 0.5, 'n_estimators': 10, 'oob_score': True}
{'max_depth': 4, 'max_features': 0.5, 'n_estimators': 7, 'oob_score': True}
{'max_depth': 3, 'max_features': 0.5, 'n_estimators': 20, 'oob_score': True}
{'max_depth': 3, 'max_features': 0.7, 'n_estimators': 20, 'oob_score': True}
{'

C:\Users\qtran\AppData\Local\Continuum\Miniconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\qtran\AppData\Local\Continuum\Miniconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\qtran\AppData\Local\Continuum\Miniconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_arg

In [35]:
# cv_result=pd.DataFrame(clf.cv_results_).sort_values('rank_test_score')
# cv_result.T

C:\Users\qtran\AppData\Local\Continuum\Miniconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\qtran\AppData\Local\Continuum\Miniconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\qtran\AppData\Local\Continuum\Miniconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_arg

,65,70,71,64,69,67,63,68,66,62,...,14,6,15,12,2,3,4,5,1,0
mean_fit_time,35.2432,28.6572,31.9064,25.9338,25.9946,14.0286,22.9574,19.8292,10.0284,17.7614,...,6.509,3.322,7.9986,3.752,2.6018,3.8334,3.9782,5.5878,1.6542,0.755
mean_score_time,0.169,0.1104,0.1282,0.1272,0.105,0.0654,0.1088,0.0931999,0.0582,0.0863999,...,0.0646,0.0429999,0.0776,0.0374,0.0798001,0.0844,0.1086,0.1208,0.0466001,0.0221999
mean_test_score,-0.808654,-0.808688,-0.808793,-0.808799,-0.809102,-0.809505,-0.809523,-0.809529,-0.809616,-0.810487,...,-0.923638,-0.92366,-0.924244,-0.924265,-0.931954,-0.931994,-0.936331,-0.937916,-0.940094,-0.952866
mean_train_score,-0.777401,-0.777813,-0.777525,-0.777442,-0.777924,-0.778238,-0.77763,-0.777819,-0.778646,-0.77772,...,-0.902799,-0.903175,-0.903436,-0.903418,-0.912226,-0.911815,-0.915699,-0.91598,-0.919761,-0.930078
param_max_depth,4,4,4,4,4,4,4,4,4,4,...,1,1,1,1,1,1,1,1,1,1
param_max_features,0.5,0.7,0.7,0.5,0.7,0.7,0.5,0.7,0.7,0.5,...,0.7,0.5,0.7,0.7,0.2,0.2,0.2,0.2,0.2,0.2
param_n_estimators,20,15,20,15,13,7,13,10,5,10,...,10,5,13,5,10,13,15,20,7,5
param_oob_score,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
params,"{'max_depth': 4, 'max_features': 0.5, 'n_estim...","{'max_depth': 4, 'max_features': 0.7, 'n_estim...","{'max_depth': 4, 'max_features': 0.7, 'n_estim...","{'max_depth': 4, 'max_features': 0.5, 'n_estim...","{'max_depth': 4, 'max_features': 0.7, 'n_estim...","{'max_depth': 4, 'max_features': 0.7, 'n_estim...","{'max_depth': 4, 'max_features': 0.5, 'n_estim...","{'max_depth': 4, 'max_features': 0.7, 'n_estim...","{'max_depth': 4, 'max_features': 0.7, 'n_estim...","{'max_depth': 4, 'max_features': 0.5, 'n_estim...",...,"{'max_depth': 1, 'max_features': 0.7, 'n_estim...","{'max_depth': 1, 'max_features': 0.5, 'n_estim...","{'max_depth': 1, 'max_features': 0.7, 'n_estim...","{'max_depth': 1, 'max_features': 0.7, 'n_estim...","{'max_depth': 1, 'max_features': 0.2, 'n_estim...","{'max_depth': 1, 'max_features': 0.2, 'n_estim...","{'max_depth': 1, 'max_features': 0.2, 'n_estim...","{'max_depth': 1, 'max_features': 0.2, 'n_estim...","{'max_depth': 1, 'max_features': 0.2, 'n_estim...","{'max_depth': 1, 'max_features': 0.2, 'n_estim..."
rank_test_score,1,2,3,4,5,6,7,8,9,10,...,63,64,65,66,67,68,69,70,71,72


# Prediction

In [46]:
params ={'max_depth': 5, 'max_features': 0.4, 'n_estimators': 25, 'oob_score': True,'random_state':1402}

In [48]:
X,y = get_X_y_ensembling(all_oof_df)

rf= RandomForestRegressor(**params)

rf.fit(X,y)

test_pred =  rf.predict(test_df)

pd.Series(test_pred).describe()

get_submission(test_pred,'ensembling_rf'); # .915 LB

C:\Users\qtran\AppData\Local\Continuum\Miniconda3\lib\site-packages\sklearn\ensemble\forest.py:724: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "


In [49]:
pd.Series(test_pred).describe()

count    214200.000000
mean          0.303533
std           0.785487
min           0.026790
25%           0.030020
50%           0.113811
75%           0.281145
max          19.404374
dtype: float64